<a href="https://colab.research.google.com/github/HandanYU/Rumour-detection/blob/main/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- replace_abbreviations
- remove url 
- get the emoji and replace them as words
- remove stop words
- keep only english letters
- stemming

In [3]:
# !pip install emoji
# !pip install nltk

  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=5a92f91dcf134fc24011694b0e97f509edb85899e2f6e4aeb666514621a3bb94
  Stored in directory: c:\users\trist\appdata\local\pip\cache\wheels\5e\8c\80\c3646df8201ba6f5070297fe3779a4b70265d0bfd961c15302
Successfully built emoji


## Convert json to dataframe

In [40]:
def json2df(json_file):
    """
    json_file: 'train_reply.json'
    """
    with open(json_file,'r+') as file:
        content = file.read()
    content = json.loads(content)#将json格式文件转化为python的字典文件
    df = pd.DataFrame(content)
    df = df.T
    return df
# df = json2df('train_reply.json')

### Sort raw data by time

In [ ]:
@timer('ms')
def sort_by_time(raw_file, json_file):
    with open(raw_file) as file:
        ids = file.readlines()

    with open(json_file, 'r+') as file:
        content = file.read()
        content=json.loads(content)
        df = pd.DataFrame(content)
        df = df.T

    save_name = raw_file[:-4] + '_sorted.txt'
    with open(save_name, 'w') as file:
        date = pd.Series(pd.DatetimeIndex(df.iloc[:, 6]), index=df.index)
        df.drop(['created_at'], axis=1, inplace=True)
        df['time'] = date

        for id_ in ids:
            ids_ = id_.strip().split(',')
            source_id = ids_[0]
            file.write(source_id)
            if len(ids_) > 1:
                reply_ids = ids_[1:]
                reply_ids[-1] = reply_ids[-1].replace('\n', '')
                valid_ids = [index for index in reply_ids if index in df.index]
                sorted_replies = df.loc[valid_ids].sort_values(by='time')
                if len(valid_ids) > 0:
                    file.write(',')

                for i, index in enumerate(sorted_replies.index):
                    file.write(index)
                    if i != len(sorted_replies.index) - 1:
                        file.write(',')

            file.write('\n')

In [ ]:
raw_files = ['train.data.txt', 'dev.data.txt']
json_files = ['./filtered data/train_reply.json', './filtered data/dev_reply.json']
for raw_file, json_file in zip(raw_files, json_files):
    sort_by_time(raw_file, json_file)

In [ ]:
with open(raw_files[0]) as file:
    train = file.readlines()
with open('train.data_sorted.txt') as file:
    s = file.readlines()
assert len(train) == len(s)

## Text pre-processing

In [125]:
import emoji
from nltk.corpus import stopwords
import re
import nltk
import json
import pandas as pd
from utils import timer

In [126]:
nltk.download('wordnet')
stemmer = nltk.stem.porter.PorterStemmer()
stopword = stopwords.words('english')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yuhandan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [127]:



@timer('ms')
def clean_tweet(content):
    from time import strftime
    # def compute_num_month(content):
    #     month_num = 0
    #     month = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "octorber", "november", "december", 
    #             "jan.", "feb.", "mar.", "apr.", "may.", "jun.", "jul.", "aug.", "sept.", "oct.", "nov.", "dec."]
    #     for i in content:
    #         if i in month:
    #             month_num += 1
    #     return month_num
    # replace_abbreviations
    content = content.lower()
    content = re.sub(r"won't", "will not", content)
    content = re.sub(r"can't", "can not", content)
    content = re.sub(r"cannot", "can not", content)
    content = re.sub(r"n't", " not", content)
    content = re.sub(r"'re", " are", content)
    content = re.sub(r"'s", " is", content)
    content = re.sub(r"'d", " would", content)
    content = re.sub(r"'ll", " will", content)
    content = re.sub(r"'t", " not", content)
    content = re.sub(r"'ve", " have", content)
    content = re.sub(r"'m", " am", content)
    # get the number of month be mentioned
    # month_num = compute_num_month(content)

    # get the number of url
    url_num = len(re.findall(r'https?://[^ ]+', content))
    url_num += len(re.findall(r'www.[^ ]+', content))
    # get the number of twitter ID be mentioned
    id_num = len(re.findall(r'@[A-Za-z0-9_]+', content))
    content = re.sub(r'@[A-Za-z0-9_]+', '', content) # remove twitter ID
    # remove url 
    ## http, https
    content = re.sub(r'https?://[^ ]+', '', content) 
    ## www.
    content = re.sub(r'www.[^ ]+', '', content)
    # get the emoji and replace them as words
    emojis = emoji.distinct_emoji_list(content)
    for e in emojis:
        content = re.sub(e, emoji.demojize(e), content)
    content = re.sub('\w+\d+\w+', '', content) # remove the word contains numbers
    content = re.sub(r'[:_.!+-=——,$%^\.\?\\~\"\'@#$%&*<>{}\[\]()/]', ' ', content) # remove punctuation
    content = re.sub(r"\s+", " ", content) # conver multiple spaces as a single space
    content = content.strip()
    # remove stop words and keep only english letters
    content = [c for c in content.split(' ') if c not in stopword and c.isalpha()]
    # do stemming
    content = [stemmer.stem(token) for token in content]
    
    return content, url_num, id_num 
    #, month_num

In [128]:
@timer('ms')
def clean_data(data_type):
  """
  Args: 
    data_type: 'dev', 'train'
  Returns:
    source_df, reply_df
  """
  with open(f'./data/full data/{data_type}_source.json', 'r+') as file:
      source_content = file.read()
  source_content = json.loads(source_content)
  source_df = pd.DataFrame(source_content)
  source_df = source_df.T
  source_df['text'] = source_df['text'].apply(lambda x: clean_tweet(x))
  
  with open(f'./data/full data/{data_type}_reply.json', 'r+') as file:
      reply_content = file.read()
  reply_content = json.loads(reply_content)
  reply_df = pd.DataFrame(reply_content)
  reply_df = reply_df.T
  reply_df['text'] = reply_df['text'].apply(lambda x: clean_tweet(x))

  return source_df, reply_df
# train_source_df, train_reply_df = clean_data('train')

# dev_source_df, dev_reply_df = clean_data('dev')


## Concat label

In [129]:
import pandas as pd
def concat_id_label(data_type, source_feature_df):
    """
    data_type: 'dev', 'train'
    """
    df = pd.DataFrame(columns=['id', 'label'])
    with open(f'./data/original_data/{data_type}_source.txt', 'r') as f:
        ids = f.readlines()
    with open(f'./data/original_data/{data_type}.label.txt', 'r') as f:
        labels = f.readlines()
    df['id'] = [id.strip() for id in ids]
    df['label'] = [label.strip() for label in labels]
    source_feature_df['id'] = source_feature_df.index
    df_labels = pd.merge(source_feature_df, df, on='id', how='left')
    return df_labels

# dev_source_df = concat_id_label('dev', dev_source_df)
# train_source_df = concat_id_label('train', train_source_df)


## Concat Sorted reply

In [130]:
def concat_reply(data_type, source_df):
    """
    data_type: 'dev', 'train'
    """
    df = pd.DataFrame(columns=['id', 'reply'])
    with open(f'./data/original_data/{data_type}.data_sorted.txt', 'r') as f:
        content = f.readlines()
    df['id'] = [c.split(',')[0].strip() for c in content]
    df['reply'] = [','.join([i.strip() for i in c.split(',')[1:]]) for c in content]
    source_df = pd.merge(source_df, df, on='id', how='left')
    return source_df

# dev_source_df = concat_reply('dev', dev_source_df)
# train_source_df = concat_reply('train', train_source_df)


In [ ]:
train_source_df, train_reply_df = clean_data('train')
train_source_df = concat_id_label('train', train_source_df)
train_source_df = concat_reply('train', train_source_df)

dev_source_df, dev_reply_df = clean_data('dev')
dev_source_df = concat_id_label('dev', dev_source_df)
dev_source_df = concat_reply('dev', dev_source_df)
